In [2]:
# Written by Chongbin Zhang. 
# Designed to facilitate UCSD HDH's calendar import from events organized in excel table. 

# necessary imports
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from datetime import datetime
from icalendar import Calendar, Event

In [3]:
# a cell for helper methods

# stringIsInt
# check if the input string can be parsed into an integer
# if so, return that integer
# else return false
# @param: strin, the input string to be checked

def stringIsInt(strin):
    if type(strin) is not str:
        raise(InputIsNotStringError())
    try:
        intout = int(strin)
        return intout
    except ValueError:
        return False
    
    
def timeProcessor(strin):
    

In [12]:
# open the excel and parse all the columns into lists
# df stands for data frame
df = pd.read_excel('ERCDeskEvents.xls', sheet_name='Sheet1')
listConfName = df['Conference Name']
listHousArea = df['College Housing Area']
listConfCord = df['Conference Coordinator']
listArrvDate = df['Arrival 1 : Date']
listArrTm_Ct = df['Arrival 1 : Time, Count']
listChkInstr = df['Arrival 1 : Check-In Instructions']
listMobi_Plc = df['Mobile Check-In Location  & Date']
    
listArrvTime = listArrCount = []

for i in range(0, 150): 
    
    # split arrival time & people count, 0~150 is ERC Desk
    temp = listArrTm_Ct[i].split(', ')
    
    # time only in these formats: 1:00pm, 11:00am, 11:00am-1:00pm
    if temp[0].find(':') == -1 and temp[0].find('a') == -1 and temp[0].find('p') == -1: # bad format
        # print("ERROR parsing: ", listConfName[i], ", because arrival time count is entered as:“", temp[0], "”Please enter this event manually. ")
        print("ERROR parsing: ", listConfName[i])
    else:
        listArrvTime.append(temp[0])
        
    if len(temp) <= 1:
        listArrCount.append(0)
    else:
        income = stringIsInt(temp[1])
        if not income:
            # print("ERROR parsing: ", listConfName[i], ", because people count is entered as:“", temp[1], "”. Please enter this event manually. ")
            print("ERROR parsing: ", listConfName[i])
        else:
            listArrCount.append(income)

    # print(listConfName[i], "\t", listArrCount[i], "\t", listArrvTime[i])
listArrTm_Ct = None # free memory


ERROR parsing:  Alexa Café by iD Tech Camps-Week 2
ERROR parsing:  Alexa Café by iD Tech Camps-Week 3
ERROR parsing:  SKYUS-Senri High School
ERROR parsing:  SKYUS-Senri High School
ERROR parsing:  TCLI Summer Leadership Program
ERROR parsing:  SKYUS-Senri High School
ERROR parsing:  SKYUS-Senri High School
ERROR parsing:  TCLI Summer Leadership Program
ERROR parsing:  Explorer Academy
ERROR parsing:  KCNA Conference
ERROR parsing:  TRIO SSSP Experience
ERROR parsing:  Explorer Academy
ERROR parsing:  Intercollegiate Teams Pre-Season Housing 1-ICA 1


In [6]:
# Calendar() to write to .ics file
cal = Calendar()

# iterate through the table to create .ics file, 150 is the last for ERC desk in this file
for i in range(3, 6):
    event = Event()

    peopleCount = listArrCount[i]
    
    try: 
        separator = listArrTm_Ct[i][0].index('-') # has a end time
        beginTime = listArrTm_Ct[i][0][0:separator].split(':')
        endTime = listArrTm_Ct[i][0][separator + 1:len(listArrTm_Ct[i][0]) + 1].split(':')
    except ValueError: # no end time b/c no '-' in arrival time: assume 1 hour long 
        begin = listArrTm_Ct[i][0].index('m')
        beginTime = listArrTm_Ct[i][0][0:(begin + 1)]
        beginTime = beginTime.split(':')
        endHour = int(beginTime[0:beginTime.index(':')]) + 1
        endTime = str(endHour) + ':' + beginTime[beginTime.index(':') + 1:len(beginTime) + 1]
            
    print('begin time: ', beginTime)
    begin = datetime(int(listArrvDate[i].year), int(listArrvDate[i].month), int(listArrvDate[i].day), 
                     int(beginTime[0]), int(beginTime[1][0:2]))
    end = datetime(int(listArrvDate[i].year), int(listArrvDate[i].month), int(listArrvDate[i].day), 
                   int(endTime[0]), int(endTime[1][0:2]))
    
    # Add people count in title
    summary = listConfName[i] + ' (' + str(peopleCount) + ')'
    
    # categorize
    if listMobi_Plc[i] == "nan": 
        summary = summary + ' (Check-in)'
    # elif (font is red): 
    #     event.add('categories, 'Check-out')
    else: 
        summary += ' (Mobile Check-in)'
    
    event.add('summary', summary)
    event.add('description', listChkInstr[i])
    event.add('dtstart', begin)
    event.add('dtend', end)
    cal.add_component(event)

    filename = './' + summary + '.ics'
    file = open(filename, 'wb')
    file.write(cal.to_ical())
    file.close()




conference name:  UC San Diego Men's Basketball Team 2


TypeError: 'NoneType' object is not subscriptable